In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, udf, lit
from pyspark.sql.types import *
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
import datetime
from pyspark.sql.types import *
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
data = [("1","Nicola","James","no",None),("2","Vaugh","Michael","31",'19990109'),
        ("3", "Pattinson","Robert","30",'fghd-01-09'),("4", "De","Maria","f43",'1997-06-09'),
        ("5","Kyle","Jen","24",'jan 01 2021')]

columns = ["Member_id", "First Name", "last Name","Age","Date"]
df = spark.createDataFrame(data = data, schema = columns)
df.show()

+---------+----------+---------+---+-----------+
Member_id|First Name|last Name|Age| Date|
+---------+----------+---------+---+-----------+
 1| Nicola| James| no| null|
 2| Vaugh| Michael| 31| 19990109|
 3| Pattinson| Robert| 30| fghd-01-09|
 4| De| Maria|f43| 1997-06-09|
 5| Kyle| Jen| 24|jan 01 2021|
+---------+----------+---------+---+-----------+

In [0]:
#Reading Common Record Format File from ADLS
#Superset Dataframe 
file_location = r"dbfs:/mnt/staging/crf_dtypes.csv"
file_type = "csv"

#Schema extraction
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","


crf_casting = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .load(file_location)

#Displaying Schema
display(crf_casting)

Column Nmae,Data Type
MEMBER_ID,integer
FIRST_NAME,string
LAST_NAME,string
MBI,integer
DOB,date
DOD,date
START_DATE,date
END_DATE,date
STATE,string
GENDER,string


In [0]:
file_location = r"dbfs:/mnt/staging/crf_dtype_demo.csv"
file_type = "csv"

#Schema extraction
infer_schema = "false"
first_row_is_header = "true"


crf_casting1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .load(file_location)

#Displaying Schema
display(crf_casting1)

MEMBER_ID,FIRST_NAME,LAST_NAME,MBI,DOB,DOD,START_DATE,END_DATE,STATE,GENDER,PHONE_NUMBER,_c11
15,y,h,9895,19980416,19980418,20160415,20160415,u,n,9765460152,null
15,y,h,9895,19980412,19980418,20160414,20160415,u,n,9765460116,null
15,y,h,9895,19981216,19980418,20160419,20160415,u,n,9765460152,null
15,y,h,9895,19980419,19980418,20160415,20160415,u,n,9765460152,null
15,y,h,9895,19970416,19980418,20160415,20160415,u,n,9765460163,null
null,null,null,null,null,null,null,null,null,null,null,null


In [0]:
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,LongType

file_location = r"dbfs:/mnt/staging/crf_dtypes.csv"
file_type = "csv"

#Schema extraction
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","


crf_casting = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .load(file_location)
crf_casting= crf_casting.withColumnRenamed("Column Nmae","column_name") \
                        .withColumnRenamed("Data Type","Data_type")

#Displaying Schema

def convertColumn(df, col_dict):
    for key, val in col_dict.items():
        if val== DateType():
             df = df.withColumn(key, to_date(df[key], "yyyyMMdd"))
        else:
             df = df.withColumn(key, col(key).cast(val))
        #df = df.withColumn(key, col(key).cast(val))
               #.withColumn('Flag_'+key, udf_validation_casting(key)) #/0 or 1 logic
    return df

def datatypeConversion(mapping):
    for key in mapping.keys():
        if mapping[key]== 'integer':
            mapping[key]=IntegerType()
        elif mapping[key]== 'string':
            mapping[key]= StringType()
        elif mapping[key]== 'date':
            mapping[key]= DateType()
        elif mapping[key]== 'long':
            mapping[key]= LongType()
            
    return mapping
        
# List of columns
Columns_ll = crf_casting1.columns
mapping = {row['column_name']:row['Data_type'] for row in crf_casting.collect()}
mapping_datatype= datatypeConversion(mapping)
# Convert the type
casted_df = convertColumn(crf_casting1, mapping_datatype)
#casted_df.printSchema()
display(casted_df)


MEMBER_ID,FIRST_NAME,LAST_NAME,MBI,DOB,DOD,START_DATE,END_DATE,STATE,GENDER,PHONE_NUMBER,_c11
15,y,h,9895,1998-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152,null
15,y,h,9895,1998-04-12,1998-04-18,2016-04-14,2016-04-15,u,n,9765460116,null
15,y,h,9895,1998-12-16,1998-04-18,2016-04-19,2016-04-15,u,n,9765460152,null
15,y,h,9895,1998-04-19,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152,null
15,y,h,9895,1997-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460163,null
null,null,null,null,null,null,null,null,null,null,null,null


In [0]:
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,LongType

file_location = r"dbfs:/mnt/staging/crf_dtypes.csv"
file_type = "csv"

#Schema extraction
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","


crf_casting = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .load(file_location)
crf_casting= crf_casting.withColumnRenamed("Column Nmae","column_name") \
                        .withColumnRenamed("Data Type","Data_type")

#Displaying Schema

def convertColumn(df, col_dict):
    for key, val in col_dict.items():
        if val== DateType():
             df = df.withColumn('casted_'+key, to_date(df[key], "yyyyMMdd"))
        else:
             df = df.withColumn('casted_'+key, col(key).cast(val))
            #.withColumn('Flag_'+key, udf_validation_casting(key))
#         df = df.withColumn('casted_'+key, df[key].cast(val))
               #.withColumn('Flag_'+key, udf_validation_casting(key)) #/0 or 1 logic
    return df

def datatypeConversion(mapping):
    for key in mapping.keys():
        if mapping[key]== 'integer':
            mapping[key]=IntegerType()
        elif mapping[key]== 'string':
            mapping[key]= StringType()
        elif mapping[key]== 'date':
            mapping[key]= DateType()
        elif mapping[key]== 'long':
            mapping[key]= LongType()
            
    return mapping
        
# List of columns
Columns_ll = crf_casting1.columns
mapping = {row['column_name']:row['Data_type'] for row in crf_casting.collect()}
mapping_datatype= datatypeConversion(mapping)
# Convert the type
casted_df1 = convertColumn(crf_casting1, mapping_datatype)
casted_df1.printSchema()
display(casted_df1)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2309521847728143> in <module> 
 47 mapping_datatype = datatypeConversion ( mapping ) 
 48 # Convert the type 
 ---> 49 casted_df1 = convertColumn ( crf_casting1 , mapping_datatype ) 
 50 casted_df1 . printSchema ( ) 
 51 display ( casted_df1 ) 

 <command-2309521847728143> in convertColumn (df, col_dict) 
 22 for key , val in col_dict . items ( ) : 
 23 if val == DateType ( ) : 
 ---> 24 df = df . withColumn ( 'casted_' + key , to_date ( df [ key ] , "yyyyMMdd" ) ) 
 25 else : 
 26 df = df . withColumn ( 'casted_' + key , col ( key ) . cast ( val ) ) 

 NameError : name 'to_date' is not defined

In [0]:
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,LongType

file_location = r"dbfs:/mnt/staging/crf_dtypes.csv"
file_type = "csv"

#Schema extraction
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","


crf_casting = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .load(file_location)
crf_casting= crf_casting.withColumnRenamed("Column Nmae","column_name") \
                        .withColumnRenamed("Data Type","Data_type")

#Displaying Schema

def convertColumn(df, col_dict):
    for key, val in col_dict.items():
        if val== DateType():
             df = df.withColumn('casted_'+key, to_date(df[key], "yyyyMMdd"))
        else:
             df = df.withColumn('casted_'+key, col(key).cast(val))
            
            
    print(key,val)
    print(col_dict)
              
    return df

def datatypeConversion(mapping):
    for key in mapping.keys():
        if mapping[key]== 'integer':
            mapping[key]=IntegerType()
        elif mapping[key]== 'string':
            mapping[key]= StringType()
        elif mapping[key]== 'date':
            mapping[key]= DateType()
        elif mapping[key]== 'long':
            mapping[key]= LongType()
            
    return mapping


  
        
# List of columns
Columns_ll = crf_casting1.columns
mapping = {row['column_name']:row['Data_type'] for row in crf_casting.collect()}
mapping_datatype= datatypeConversion(mapping)
# Convert the type
casted_df1 = convertColumn(crf_casting1, mapping_datatype)

casted_df1.printSchema()
display(casted_df1)


PHONE_NUMBER LongType
{'MEMBER_ID': IntegerType, 'FIRST_NAME': StringType, 'LAST_NAME': StringType, 'MBI': IntegerType, 'DOB': DateType, 'DOD': DateType, 'START_DATE': DateType, 'END_DATE': DateType, 'STATE': StringType, 'GENDER': StringType, 'PHONE_NUMBER': LongType}
root
-- MEMBER_ID: string (nullable = true)
-- FIRST_NAME: string (nullable = true)
-- LAST_NAME: string (nullable = true)
-- MBI: string (nullable = true)
-- DOB: string (nullable = true)
-- DOD: string (nullable = true)
-- START_DATE: string (nullable = true)
-- END_DATE: string (nullable = true)
-- STATE: string (nullable = true)
-- GENDER: string (nullable = true)
-- PHONE_NUMBER: string (nullable = true)
-- _c11: string (nullable = true)
-- casted_MEMBER_ID: integer (nullable = true)
-- casted_FIRST_NAME: string (nullable = true)
-- casted_LAST_NAME: string (nullable = true)
-- casted_MBI: integer (nullable = true)
-- casted_DOB: date (nullable = true)
-- casted_DOD: date (nullable = true)
-- casted_START_DATE: date (nullable = true)
-- casted_END_DATE: date (nullable = true)
-- casted_STATE: string (nullable = true)
-- casted_GENDER: string (nullable = true)
-- casted_PHONE_NUMBER: long (nullable = true)

MEMBER_ID,FIRST_NAME,LAST_NAME,MBI,DOB,DOD,START_DATE,END_DATE,STATE,GENDER,PHONE_NUMBER,_c11,casted_MEMBER_ID,casted_FIRST_NAME,casted_LAST_NAME,casted_MBI,casted_DOB,casted_DOD,casted_START_DATE,casted_END_DATE,casted_STATE,casted_GENDER,casted_PHONE_NUMBER
15,y,h,9895,19980416,19980418,20160415,20160415,u,n,9765460152,null,15,y,h,9895,1998-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152
15,y,h,9895,19980412,19980418,20160414,20160415,u,n,9765460116,null,15,y,h,9895,1998-04-12,1998-04-18,2016-04-14,2016-04-15,u,n,9765460116
15,y,h,9895,19981216,19980418,20160419,20160415,u,n,9765460152,null,15,y,h,9895,1998-12-16,1998-04-18,2016-04-19,2016-04-15,u,n,9765460152
15,y,h,9895,19980419,19980418,20160415,20160415,u,n,9765460152,null,15,y,h,9895,1998-04-19,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152
15,y,h,9895,19970416,19980418,20160415,20160415,u,n,9765460163,null,15,y,h,9895,1997-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460163
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [0]:
from pyspark.sql.functions import *
casted_df12=["DOB","DOD","START_DATE","END_DATE"]
print(casted_df12)

['DOB', 'DOD', 'START_DATE', 'END_DATE']

In [0]:
from pyspark.sql import functions as F
def dateStringFormatter(column):
    return str(column)[:4] + "-" + str(column)[4:6] + "-" +str(column)[6:8]
#         return F.to_date(F.col(a),'yyyyMMdd')

In [0]:
from pyspark.sql.functions import col


# stringFormatterUDF = udf(lambda z:dateStringFormatter(z))
# mbr_load1= casted_df12.withColumn("DOB", stringFormatterUDF(col('DOB'))) \
mbr_load1= casted_df1.withColumn("DOB", dateStringFormatter('DOB')) \
.withColumn("DOD",dateStringFormatter('DOD')) \
.withColumn("START_DATE",dateStringFormatter('START_DATE')) \
.withColumn("END_DATE",dateStringFormatter('END_DATE')) \


display(mbr_load1)


--------------------------------------------------------------------------- 
 AssertionError Traceback (most recent call last)
 <command-1368553124595576> in <module> 
 4 # stringFormatterUDF = udf(lambda z:dateStringFormatter(z)) 
 5 # mbr_load1= casted_df12.withColumn("DOB", stringFormatterUDF(col('DOB'))) \ 
 ----> 6 mbr_load1 = casted_df1 . withColumn ( "DOB" , dateStringFormatter ( 'DOB' ) ) \ 
 7 . withColumn ( "DOD" , dateStringFormatter ( 'DOD' ) ) \ 
 8 . withColumn ( "START_DATE" , dateStringFormatter ( 'START_DATE' ) ) \ 

 /databricks/spark/python/pyspark/sql/dataframe.py in withColumn (self, colName, col) 
 2475 
 2476 """
 -> 2477 assert isinstance ( col , Column ) , "col should be Column" 
 2478 return DataFrame ( self . _jdf . withColumn ( colName , col . _jc ) , self . sql_ctx ) 
 2479 

 AssertionError : col should be Column

In [0]:
from pyspark.sql.functions import col


stringFormatterUDF = udf(lambda z:dateStringFormatter(z))
mbr_load1= casted_df1.withColumn("DOB", stringFormatterUDF(col('DOB'))) \
.withColumn("DOD",stringFormatterUDF("DOD")) \
.withColumn("START_DATE",stringFormatterUDF("START_DATE")) \
.withColumn("END_DATE",stringFormatterUDF("END_DATE"))
display(mbr_load1)

MEMBER_ID,FIRST_NAME,LAST_NAME,MBI,DOB,DOD,START_DATE,END_DATE,STATE,GENDER,PHONE_NUMBER,_c11,casted_MEMBER_ID,casted_FIRST_NAME,casted_LAST_NAME,casted_MBI,casted_DOB,casted_DOD,casted_START_DATE,casted_END_DATE,casted_STATE,casted_GENDER,casted_PHONE_NUMBER
15,y,h,9895,1998-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152,null,15,y,h,9895,1998-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152
15,y,h,9895,1998-04-12,1998-04-18,2016-04-14,2016-04-15,u,n,9765460116,null,15,y,h,9895,1998-04-12,1998-04-18,2016-04-14,2016-04-15,u,n,9765460116
15,y,h,9895,1998-12-16,1998-04-18,2016-04-19,2016-04-15,u,n,9765460152,null,15,y,h,9895,1998-12-16,1998-04-18,2016-04-19,2016-04-15,u,n,9765460152
15,y,h,9895,1998-04-19,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152,null,15,y,h,9895,1998-04-19,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152
15,y,h,9895,1997-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460163,null,15,y,h,9895,1997-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460163
null,null,null,null,None--,None--,None--,None--,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [0]:
display(casted_df1)

MEMBER_ID,FIRST_NAME,LAST_NAME,MBI,DOB,DOD,START_DATE,END_DATE,STATE,GENDER,PHONE_NUMBER,_c11,casted_MEMBER_ID,casted_FIRST_NAME,casted_LAST_NAME,casted_MBI,casted_DOB,casted_DOD,casted_START_DATE,casted_END_DATE,casted_STATE,casted_GENDER,casted_PHONE_NUMBER
15,y,h,9895,19980416,19980418,20160415,20160415,u,n,9765460152,null,15,y,h,9895,1998-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152
15,y,h,9895,19980412,19980418,20160414,20160415,u,n,9765460116,null,15,y,h,9895,1998-04-12,1998-04-18,2016-04-14,2016-04-15,u,n,9765460116
15,y,h,9895,19981216,19980418,20160419,20160415,u,n,9765460152,null,15,y,h,9895,1998-12-16,1998-04-18,2016-04-19,2016-04-15,u,n,9765460152
15,y,h,9895,19980419,19980418,20160415,20160415,u,n,9765460152,null,15,y,h,9895,1998-04-19,1998-04-18,2016-04-15,2016-04-15,u,n,9765460152
15,y,h,9895,19970416,19980418,20160415,20160415,u,n,9765460163,null,15,y,h,9895,1997-04-16,1998-04-18,2016-04-15,2016-04-15,u,n,9765460163
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [0]:
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,LongType

file_location = r"dbfs:/mnt/staging/crf_dtypes.csv"
file_type = "csv"

#Schema extraction
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","


crf_casting = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .load(file_location)
crf_casting= crf_casting.withColumnRenamed("Column Nmae","column_name") \
                        .withColumnRenamed("Data Type","Data_type")

#Displaying Schema

def convertColumn(df, col_dict):
    for key, val in col_dict.items():
        if val== DateType():
             df = df.withColumn('casted_'+key, to_date(df[key], "yyyyMMdd"))
        elif val == IntegerType():
                df = df.withColumn('casted_'+key, col(key).cast(val))
            
             #df = df.withColumn('casted_'+key, col(key).cast(val))
         #df = df.withColumn('casted_'+key, df[key].cast(val))
               #.withColumn('Flag_'+key, udf_validation_casting(key)) #/0 or 1 logic
    return df

# def datatypeConversion(mapping):
#     for key in mapping.keys():
#         if mapping[key]== 'integer':
#             mapping[key]=IntegerType()
#         elif mapping[key]== 'string':
#             mapping[key]= StringType()
#         elif mapping[key]== 'date':
#             mapping[key]= DateType()
#         elif mapping[key]== 'long':
#             mapping[key]= LongType()
            
#     return mapping
        
# List of columns
Columns_ll = crf_casting1.columns
mapping = {row['column_name']:row['Data_type'] for row in crf_casting.collect()}
#mapping_datatype= datatypeConversion(mapping)
# Convert the type
casted_df1 = convertColumn(crf_casting1,mapping )
casted_df1.printSchema()
display(casted_df1)

root
-- MEMBER_ID: string (nullable = true)
-- FIRST_NAME: string (nullable = true)
-- LAST_NAME: string (nullable = true)
-- MBI: string (nullable = true)
-- DOB: string (nullable = true)
-- DOD: string (nullable = true)
-- START_DATE: string (nullable = true)
-- END_DATE: string (nullable = true)
-- STATE: string (nullable = true)
-- GENDER: string (nullable = true)
-- PHONE_NUMBER: string (nullable = true)
-- _c11: string (nullable = true)

MEMBER_ID,FIRST_NAME,LAST_NAME,MBI,DOB,DOD,START_DATE,END_DATE,STATE,GENDER,PHONE_NUMBER,_c11
15,y,h,9895,19980416,19980418,20160415,20160415,u,n,9765460152,null
15,y,h,9895,19980412,19980418,20160414,20160415,u,n,9765460116,null
15,y,h,9895,19981216,19980418,20160419,20160415,u,n,9765460152,null
15,y,h,9895,19980419,19980418,20160415,20160415,u,n,9765460152,null
15,y,h,9895,19970416,19980418,20160415,20160415,u,n,9765460163,null
null,null,null,null,null,null,null,null,null,null,null,null
